In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.current_device()

0

In [4]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1650 Ti'

In [5]:
torch.cuda.memory_allocated()

0

In [6]:
torch.cuda.memory_cached()

D:\ARNAV\Jupyter notebook\Pytorch\envpytorch\lib\site-packages\torch\cuda\memory.py:440: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


0

In [7]:
var1 = torch.FloatTensor([1.0, 2.0, 3.0])

In [8]:
var1

tensor([1., 2., 3.])

In [9]:
var1.device

device(type='cpu')

#### For converting the above cpu device to cuda for the variable var1 we will do the following small changes:

In [10]:
var1 = torch.FloatTensor([1.0, 2.0, 3.0]).cuda()

In [11]:
var1

tensor([1., 2., 3.], device='cuda:0')

In [12]:
var1.device

device(type='cuda', index=0)

#### Example based on Pima Diabetes prediction

In [18]:
import pandas as pd
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [14]:
X = df.drop('Outcome', axis = 1).values #Independent features
y = df['Outcome'].values #Dependent features

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

In [16]:
# Importing necessary libraries from Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F

In [17]:
#### Creating Tensors
X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()
y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()

In [19]:
X_train.device

device(type='cuda', index=0)

In [20]:
#### Creating model with Pytorch

class ANN_model(nn.Module): #we have to inherit nn library for creating models
    def __init__(self, input_features = 8, hidden1 = 20, hidden2 = 20, out_features = 2):
        '''
        input_features = size of the independent variables
        hidden1 = total nodes in the first hidden layer
        hidden2 = total nodes in the second hidden layer
        '''
        super().__init__() # inheriting the parent class - nn.Module
        self.f_connected1 = nn.Linear(input_features, hidden1) # connecting input features to 1st hidden layer fully connected layer
        self.f_connected2 = nn.Linear(hidden1, hidden2) # connecting 1st hidden layer to 2nd hidden layer
        self.out = nn.Linear(hidden2, out_features) # connecting 2nd hidden layer to the output layer
        
    def forward(self, x):
        '''
        x = track the gradient descent and all the back propogation
        '''
        x = F.relu(self.f_connected1(x))
        x = F.relu(self.f_connected2(x))
        x = self.out(x)
        return x

In [22]:
### instantiate my ANN_model
torch.manual_seed(20)
model=ANN_model()

In [23]:
model.parameters

<bound method Module.parameters of ANN_model(
  (f_connected1): Linear(in_features=8, out_features=20, bias=True)
  (f_connected2): Linear(in_features=20, out_features=20, bias=True)
  (out): Linear(in_features=20, out_features=2, bias=True)
)>

In [25]:
for i in model.parameters():
    print(i.is_cuda)

False
False
False
False
False
False


In [26]:
# Now changing the model to cuda
model = model.cuda()

In [27]:
for i in model.parameters():
    print(i.is_cuda)

True
True
True
True
True
True


In [28]:
###Backward Propogation-- Define the loss_function,define the optimizer
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

In [30]:
# Now checking the cuda memory and cache memory allocated
torch.cuda.memory_allocated()

37376

In [31]:
torch.cuda.memory_cached()

D:\ARNAV\Jupyter notebook\Pytorch\envpytorch\lib\site-packages\torch\cuda\memory.py:440: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


2097152

In [32]:
import time
start_time=time.time()
epochs=10000
final_losses=[]
for i in range(epochs):
    i = i + 1
    y_pred=model.forward(X_train)
    loss=loss_function(y_pred,y_train)
    final_losses.append(loss)
    if i%10==1:
        print("Epoch number: {} and the loss : {}".format(i,loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
print(time.time()-start_time)

Epoch number: 1 and the loss : 3.4572105407714844
Epoch number: 11 and the loss : 0.8019208908081055
Epoch number: 21 and the loss : 0.6090319752693176
Epoch number: 31 and the loss : 0.5917771458625793
Epoch number: 41 and the loss : 0.5679707527160645
Epoch number: 51 and the loss : 0.5529042482376099
Epoch number: 61 and the loss : 0.5410094857215881
Epoch number: 71 and the loss : 0.5310389995574951
Epoch number: 81 and the loss : 0.5220361351966858
Epoch number: 91 and the loss : 0.5135972499847412
Epoch number: 101 and the loss : 0.5061255097389221
Epoch number: 111 and the loss : 0.49834081530570984
Epoch number: 121 and the loss : 0.49605533480644226
Epoch number: 131 and the loss : 0.4828636944293976
Epoch number: 141 and the loss : 0.4755900204181671
Epoch number: 151 and the loss : 0.48198413848876953
Epoch number: 161 and the loss : 0.48065462708473206
Epoch number: 171 and the loss : 0.4706929326057434
Epoch number: 181 and the loss : 0.45911404490470886
Epoch number: 191 